# 🚀 OpenClaw + Dolphin 3 - Google Colab

Sansürsüz AI Sunucusu - Colab'da Çalıştır

## 📋 Adımlar

1. **Runtime → Change runtime type → T4 GPU** seç
2. Tüm hücreleri sırayla çalıştır (Runtime → Run all)
3. Ngrok token'ını gir
4. Public URL'yi kopyala
5. Tarayıcıda aç ve kullan!

## ⏱️ Kurulum Süresi: ~15-20 dakika

In [ ]:
# 1. Sistem kontrolü ve paket kurulumu
import sys
import subprocess

print("="*70)
print("ADIM 1: Sistem Kontrolü ve Paket Kurulumu")
print("="*70)

# GPU kontrolü
print("\n🔍 GPU kontrolü...")
!nvidia-smi --query-gpu=name,memory.total --format=csv,noheader

print("\n📦 Paketler kuruluyor (5-10 dakika)...")
!pip install -q vllm flask pyngrok requests

print("\n✅ Paketler kuruldu!")

In [ ]:
# 2. app.py dosyasını indir ve kontrol et
print("="*70)
print("ADIM 2: Uygulama Dosyası İndiriliyor")
print("="*70)

import os
import requests

print("\n📥 app.py indiriliyor...")
url = "https://raw.githubusercontent.com/SuperHelix77/openclaw-dolphin3/main/app.py"
response = requests.get(url)

if response.status_code == 200:
    with open('app.py', 'w') as f:
        f.write(response.text)
    print("✅ app.py indirildi")
    print(f"📊 Dosya boyutu: {len(response.text)} bytes")
else:
    print(f"❌ Hata: {response.status_code}")
    sys.exit(1)

In [ ]:
# 3. Ngrok token al ve yapılandır
print("="*70)
print("ADIM 3: Ngrok Yapılandırması")
print("="*70)

from pyngrok import ngrok
import getpass

print("\n🔑 Ngrok token gerekli")
print("👉 https://dashboard.ngrok.com/get-started/your-authtoken")
print("")

ngrok_token = getpass.getpass("Ngrok token'ını yapıştır: ")

try:
    ngrok.set_auth_token(ngrok_token)
    print("✅ Ngrok token ayarlandı")
except Exception as e:
    print(f"❌ Ngrok token hatası: {e}")
    sys.exit(1)

In [ ]:
# 4. Flask sunucusunu başlat
print("="*70)
print("ADIM 4: Flask Sunucusu Başlatılıyor")
print("="*70)

import subprocess
import time
import threading

print("\n🚀 Flask sunucusu başlatılıyor...")
print("⏳ Model yükleniyor (10-15 dakika sürebilir)...")
print("")
print("💡 İpucu: Model indirme ilerlemesini görmek için")
print("   başka bir hücrede 'tail -f nohup.out' çalıştırabilirsin")

# Flask'ı arka planda başlat
process = subprocess.Popen(
    ['python', 'app.py'],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True
)

# Sunucunun başlaması için bekle
print("\n⏳ Sunucu başlatılıyor (30 saniye)...")
time.sleep(30)

print("✅ Flask sunucusu başlatıldı (arka planda çalışıyor)")

In [ ]:
# 5. Sunucu durumunu kontrol et
print("="*70)
print("ADIM 5: Sunucu Durumu Kontrolü")
print("="*70)

import time
import requests

print("\n🔍 Sunucu hazır olana kadar bekleniyor...")
print("⏳ Model yüklenmesi devam ediyor (10-15 dakika)...")

max_attempts = 60  # 60 * 30 saniye = 30 dakika
attempt = 0

while attempt < max_attempts:
    try:
        response = requests.get('http://localhost:8000/health', timeout=5)
        if response.status_code == 200:
            print("\n✅ Sunucu hazır!")
            break
    except:
        pass
    
    attempt += 1
    if attempt % 2 == 0:  # Her 60 saniyede bir güncelle
        print(f"⏳ Bekleniyor... ({attempt * 30} saniye geçti)")
    time.sleep(30)

if attempt >= max_attempts:
    print("\n⚠️  Sunucu başlatılamadı. Logs'u kontrol et.")
else:
    print("\n🎉 Model yüklendi ve sunucu hazır!")

In [ ]:
# 6. Public URL oluştur
print("="*70)
print("ADIM 6: Public URL Oluşturuluyor")
print("="*70)

from pyngrok import ngrok

print("\n🌐 Public URL oluşturuluyor...")
public_url = ngrok.connect(8000)

print("\n" + "="*70)
print("✅ BAŞARILI! Sunucu hazır ve erişilebilir.")
print("="*70)
print(f"\n🌍 Public URL: {public_url}")
print("\n📝 Bu URL'yi tarayıcıda aç ve kullan!")
print("\n⚠️  Not: Colab sekmesini AÇIK TUT, yoksa sunucu kapanır.")
print("="*70)

# URL'yi dosyaya kaydet
with open('public_url.txt', 'w') as f:
    f.write(str(public_url))
print("\n💾 URL 'public_url.txt' dosyasına kaydedildi")

In [ ]:
# 7. Sunucu loglarını göster (isteğe bağlı)
print("="*70)
print("SUNUCU LOGLARI (Son 50 satır)")
print("="*70)
print("")

# Process çıktısını oku
import subprocess
try:
    result = subprocess.run(
        ['ps', 'aux'],
        capture_output=True,
        text=True
    )
    print("Çalışan Python süreçleri:")
    for line in result.stdout.split('\n'):
        if 'python' in line.lower():
            print(line)
except:
    print("Log gösterilemedi")

print("\n💡 İpucu: Sunucu çalışmaya devam ediyor.")
print("   Bu hücreyi tekrar çalıştırarak durumu kontrol edebilirsin.")

In [ ]:
# 8. Keep-alive (Sunucuyu açık tut)
print("📊 Sunucu çalışıyor...")
print("⚠️  Bu hücreyi çalıştırırsan Colab oturumu açık kalır.")
print("   Durdurmak için: Runtime → Interrupt execution")
print("="*70)

import time
try:
    while True:
        time.sleep(60)
        print(".", end="", flush=True)
except KeyboardInterrupt:
    print("\n\n⏹️  Durduruldu")

## 🎉 Tamamlandı!

Artık web arayüzünü kullanabilirsin:

1. Yukarıdaki **Public URL**'yi kopyala
2. Yeni bir tarayıcı sekmesinde aç
3. Chat arayüzünü kullan!

## 📱 OpenClaw Config

Yerel makinende `~/.openclaw/config.yaml`:

```yaml
models:
  default:
    provider: openai
    model: dolphin-2.9-llama3.1-8b
    credentials:
      apiKey: 'ollama'
      apiBase: 'YOUR_PUBLIC_URL_HERE'
```

## ⚠️ Önemli Notlar

- Colab sekmesini **AÇIK TUT**
- 12 saat sonra otomatik kapanır
- GPU kullanımı sınırlıdır
- Ücretsiz ve sansürsüz! 🎉

## 🐛 Sorun Giderme

### Sunucu başlamadı
- GPU seçili mi kontrol et
- Runtime'ı yeniden başlat

### Model yüklenmiyor
- 15-20 dakika bekle
- Logs'u kontrol et

### Ngrok hatası
- Token'ı doğru kopyaladın mı?
- Ngrok hesabı aktif mi?